In [1]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

In [2]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))


In [3]:
people = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)

dataset = list()
for person in people:
    
    playlist = spotify.playlist(person["Link"])
    songs = spotify.playlist_items(playlist["id"])
    #print(len(songs))
    aux = set()
    
    for i in range (len(songs["items"])):
        try: 
            genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres']
            for genre in genres:
                aux.add(genre)
            dataset.append(aux)
        except:
            continue   
#print(dataset)

In [4]:
grafo = nx.Graph()

for genres in dataset:
    for genre1 in genres:
        for genre2 in genres:
            grafo.add_node(genre1)
            if genre1 != genre2 and not grafo.has_edge(genre2, genre1):
                grafo.add_edge(genre1, genre2)
nx.write_gexf(grafo, 'global_genres14.gexf')
with open('global_genres14.pkl', 'wb') as f:
    pickle.dump(grafo, f)